# Cartilage Generation 
This notebook explains a direct method to generate the lower body cartilage surface mesh based on the bone geometries
    

## imports and user-defined properties

In [1]:
import numpy as np
import meshplot as mp
import os
import igl
import time
from pathlib import Path
import sys
import pandas as pd

sys.path.append('../')

import src

INPUT SUBJECT INFORMATION:

- model_id: choose between m1 -> m11 to select one of the detaset in the repository.
- i_dim, o_dim: the input and output dimension ( 'mm' = millimeters, 'm' = meters )
- o_format: the format you want the files to be save at ( '.obj' , '.stl' )

In [2]:
# model id
model_id = 'm1'

# do you want full or implicit models of the cartilage models?
full_model = True

# dimensions 
i_dim  = 'mm'     
o_dim  = 'mm'

# suffix
i_format = '.obj'
o_format = '.obj'


DIRECTORIES and PATHS:
- main_dir: locate this directory to where the libhip repository is cloned to.
- i_dir:  the directory of the input bone surface mesh files.
- o_dir:  the directory of the output bone and cartilage model results.
- anatomical_dir: the directory of the anatomical measurements of the bone and cartilage models.
- config_path: path to the subject-specific parameters to create cartilage models.


In [3]:
main_dir = Path('..')

i_dir     = main_dir/ 'model_repository'/ 'CleanSegment'/ model_id
o_dir     = main_dir/ 'model_generation'/ 'cargen_output'/ model_id
nodal_o_dir  = main_dir/ 'model_generation'/ 'mid_outputs'/ 'nodal_output' / model_id

anatomical_dir = main_dir/ 'model_generation'/ 'anatomical_info'
model_name   = '/'+ model_id + '_jnt_ant.csv'
model_anatomical_path = str(anatomical_dir) + model_name
    
config_name  = model_id + '_config'
config_path  = str((main_dir/ 'config'/ config_name).with_suffix('.yml'))
config       = src.Config (config_path)

# Remove all files inside output directory if it exists, otherwise create it
if o_dir.is_dir():
    for file in o_dir.iterdir():
        if file.is_file():
            file.unlink()
else:
    o_dir.mkdir(exist_ok=False)
#    
if nodal_o_dir.is_dir():
    for file in nodal_o_dir.iterdir():
        if file.is_file():
            file.unlink()
else:
    nodal_o_dir.mkdir(exist_ok=False)
# 
if os.path.exists(model_anatomical_path):
    os.remove(model_anatomical_path)

In [4]:
# save the model id to the relative anatomincal information file
df = pd.read_csv (str(anatomical_dir) + '/default_ant.csv', encoding = 'utf-8')
df.loc[1, 'Value'] = model_id
df.to_csv(model_anatomical_path, index=False)

In [5]:
# bone input paths
i_bone_tag = model_id + '_clean_'

i_sacrum  = i_bone_tag + 'sacrum_'  + i_dim
i_lpelvis = i_bone_tag + 'lpelvis_' + i_dim
i_rpelvis = i_bone_tag + 'rpelvis_' + i_dim
i_lfemur  = i_bone_tag + 'lfemur_'  + i_dim
i_rfemur  = i_bone_tag + 'rfemur_'  + i_dim

i_sacrum_path  = str((i_dir/ i_sacrum).with_suffix(i_format))
i_lpelvis_path = str((i_dir/ i_lpelvis).with_suffix(i_format))
i_rpelvis_path = str((i_dir/ i_rpelvis).with_suffix(i_format))
i_lfemur_path  = str((i_dir/ i_lfemur).with_suffix(i_format))
i_rfemur_path  = str((i_dir/ i_rfemur).with_suffix(i_format))


In [6]:
# bone output paths
o_bone_tag = model_id + '_cg_bone_'

o_sacrum  = o_bone_tag + 'sacrum'
o_lpelvis = o_bone_tag + 'lpelvis'
o_rpelvis = o_bone_tag + 'rpelvis' 
o_lfemur  = o_bone_tag + 'lfemur' 
o_rfemur  = o_bone_tag + 'rfemur'

o_sacrum_path  = str((o_dir/ o_sacrum).with_suffix(o_format))
o_lpelvis_path = str((o_dir/ o_lpelvis).with_suffix(o_format))
o_rpelvis_path = str((o_dir/ o_rpelvis).with_suffix(o_format))
o_lfemur_path  = str((o_dir/ o_lfemur).with_suffix(o_format))
o_rfemur_path  = str((o_dir/ o_rfemur).with_suffix(o_format))


In [7]:
# joint output paths
if full_model:
    o_joint_tag = model_id + '_cg_jnt_'
else:
    o_joint_tag = model_id + '_cg_jnt_implicit_'
    
# sacroiliac joint
lsj = o_joint_tag + 'lsi'
rsj = o_joint_tag + 'rsi'

# pubic symphysis
pj  = o_joint_tag + 'ps'

# sacroiliac and pubic joint paths
o_lsj_path = str((o_dir/ lsj).with_suffix(o_format))
o_rsj_path = str((o_dir/ rsj).with_suffix(o_format))
o_pj_path  = str((o_dir/ pj).with_suffix(o_format))

# hip joint with gap
lhj_ac_wg = o_joint_tag + 'lh_ac_wg' 
lhj_fc_wg = o_joint_tag + 'lh_fc_wg' 
rhj_ac_wg = o_joint_tag + 'rh_ac_wg' 
rhj_fc_wg = o_joint_tag + 'rh_fc_wg'  

# hip joint without gap
lhj_ac_wog = o_joint_tag + 'lh_ac_wog' 
lhj_fc_wog = o_joint_tag + 'lh_fc_wog'
rhj_ac_wog = o_joint_tag + 'rh_ac_wog'
rhj_fc_wog = o_joint_tag + 'rh_fc_wog'

# hip joint w/wo gap paths
o_lhj_ac_wg_path = str((o_dir/ lhj_ac_wg).with_suffix(o_format))
o_rhj_ac_wg_path = str((o_dir/ rhj_ac_wg).with_suffix(o_format))
o_lhj_fc_wg_path = str((o_dir/ lhj_fc_wg).with_suffix(o_format))
o_rhj_fc_wg_path = str((o_dir/ rhj_fc_wg).with_suffix(o_format))

o_lhj_ac_wog_path = str((o_dir/ lhj_ac_wog).with_suffix(o_format))
o_rhj_ac_wog_path = str((o_dir/ rhj_ac_wog).with_suffix(o_format))
o_lhj_fc_wog_path = str((o_dir/ lhj_fc_wog).with_suffix(o_format))
o_rhj_fc_wog_path = str((o_dir/ rhj_fc_wog).with_suffix(o_format))


## implementation

In [8]:
s_vertices, s_faces  = src.read (i_sacrum_path, i_dim) 
lp_vertices,lp_faces = src.read (i_lpelvis_path, i_dim)
rp_vertices,rp_faces = src.read (i_rpelvis_path, i_dim) 
lf_vertices,lf_faces = src.read (i_lfemur_path, i_dim)
rf_vertices,rf_faces = src.read (i_rfemur_path, i_dim) 

number of faces after reading 46726
number of faces after reading 35890
number of faces after reading 36000
number of faces after reading 42782
number of faces after reading 39022


In [9]:
frame = mp.plot(s_vertices, s_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (lp_vertices, lp_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rp_vertices, rp_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (lf_vertices, lf_faces, c = src.bone, shading = src.sh_true)
frame.add_mesh (rf_vertices, rf_faces, c = src.bone, shading = src.sh_true)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.207897…

4

### timer

In [10]:
# start the timer
then = time.time()

### left hip joint - acetabular cartilage (with  and without gap)

In [11]:
config_lhj_ac = config.lhj_ac_var

if not full_model:
    config_lhj_ac.full_model = False
    

In [12]:
smooth_lp_vertices, lhj_ac_vertices_wg, lhj_ac_faces_wg, lhj_ac_vertices_wog, lhj_ac_faces_wog, lfc_face_idxs = src.get_hj_ac(lp_vertices, lp_faces,
                                                                                                                              lf_vertices, lf_faces,
                                                                                                                              config_lhj_ac, model_anatomical_path)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(60.993045…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(60.993045…

minimum thickness in the initial layer with gap 0.88066
minimum thickness in the initial layer without gap 0.97851


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(60.993045…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(60.993045…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(73.529464…

minimum cartilage thickness w/wo gap is:  0.38 / 0.42


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(101.58516…

### left hip joint - femoral cartilage (with gap and without gap)

In [13]:
config_lhj_fc = config.lhj_fc_var

if not full_model:
    config_lhj_fc.full_model = False

In [14]:
# make it
smooth_lf_vertices, lhj_fc_vertices_wg, lhj_fc_faces_wg, lhj_fc_vertices_wog, lhj_fc_faces_wog = src.get_hj_fc (lf_vertices, lf_faces, 
                                                                                                                lp_vertices, lp_faces, 
                                                                                                                lfc_face_idxs, config_lhj_fc, 
                                                                                                                model_anatomical_path, nodal_o_dir)



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(101.58516…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(101.58516…

minimum thickness in the initial layer with gap 0.85995
minimum thickness in the initial layer without gap 0.9555


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(101.58516…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(101.58516…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(77.309226…

### measure the gap between the two HJ cartilages

In [15]:
sd_value, _, closest_points = igl.signed_distance(lhj_fc_vertices_wg, lhj_ac_vertices_wg, lhj_ac_faces_wg, return_normals=False)

print('minimum distance in the left hip joint',np.round(np.min(sd_value),3))

# show the penetration
index = np.where (sd_value < 0 )[0]

frame = mp.plot(lhj_ac_vertices_wg, lhj_ac_faces_wg, c=src.pastel_orange, shading = src.sh_true)
frame.add_mesh (lhj_fc_vertices_wg, lhj_fc_faces_wg, c = src.pastel_yellow, shading = src.sh_true)

if len(index)!= 0 :
    frame.add_points(lhj_fc_vertices_wg[index], shading={"point_color": "green", "point_size": 20})
    

df = pd.read_csv(str(model_anatomical_path), encoding='utf-8')
df.loc[10, 'Value'] = np.round(np.min(sd_value),3)

df.to_csv(str(model_anatomical_path), index=False)

minimum distance in the left hip joint 0.108


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(73.529464…

### remove penetration in the "without gap" version

In [16]:
lhj_fc_vertices_wog = src.remove_penetration(lhj_fc_vertices_wog, lhj_ac_vertices_wog, lhj_ac_faces_wog)

### export results

In [17]:
src.save_surface (lhj_fc_vertices_wg, lhj_fc_faces_wg, o_dim, o_lhj_fc_wg_path)
src.save_surface (lhj_ac_vertices_wg, lhj_ac_faces_wg, o_dim, o_lhj_ac_wg_path)

src.save_surface (lhj_fc_vertices_wog, lhj_fc_faces_wog, o_dim, o_lhj_fc_wog_path)
src.save_surface (lhj_ac_vertices_wog, lhj_ac_faces_wog, o_dim, o_lhj_ac_wog_path)


### left SI joint

In [18]:
config_lsj = config.lsj_var

if not full_model:
    config_lsj.full_model = False

In [19]:
smooth_s_vertices, smooth_lp_vertices, lsj_vertices, lsj_faces = src.get_sj(s_vertices, s_faces,
                                                                           smooth_lp_vertices, lp_faces,
                                                                           config_lsj, model_anatomical_path)

# export results
src.save_surface (lsj_vertices, lsj_faces, o_dim, o_lsj_path)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.207897…

Primary side: max dihedral angle before smoothing is  0.72 radians ( 41.25 degrees).
Primary side: max dihedral angle after smoothing is  0.73 radians ( 41.83 degrees).

Quality control results for the primary side: 
Everything is clean in the primary interface. We will now continue to the secondary interface:



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.207897…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(60.993045…

Secondary layer: max dihedral angle before smoothing is  0.77 radians ( 44.12 degrees).
Secondary layer: max dihedral angle after smoothing is  0.79 radians ( 45.26 degrees).
Quality control results for the base layer: 

Everything is clean in the base layer. We will now continue to create the wall:



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(60.993045…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.207897…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(60.993045…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(39.462299…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(39.462299…

### right hip joint - acetabular cartilage (with and without gap)

In [20]:
config_rhj_ac = config.rhj_ac_var

if not full_model:
    config_rhj_ac.full_model = False

In [21]:
smooth_rp_vertices, rhj_ac_vertices_wg, rhj_ac_faces_wg, rhj_ac_vertices_wog, rhj_ac_faces_wog, rfc_face_idxs = src.get_hj_ac( rp_vertices, rp_faces, rf_vertices, rf_faces, config_rhj_ac, model_anatomical_path )



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-68.46923…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-68.46923…

minimum thickness in the initial layer with gap 0.94745
minimum thickness in the initial layer without gap 1.05272


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-68.46923…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-68.46923…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-86.66569…

minimum cartilage thickness w/wo gap is:  0.43 / 0.47


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-116.6246…

### right hip joint - femoral cartilage ( with and without gap )

In [22]:
config_rhj_fc = config.rhj_fc_var

if not full_model:
    config_rhj_fc.full_model = False

In [23]:
# make it
smooth_rf_vertices, rhj_fc_vertices_wg, rhj_fc_faces_wg, rhj_fc_vertices_wog, rhj_fc_faces_wog = src.get_hj_fc (rf_vertices, rf_faces, 
                                                                                                                rp_vertices, rp_faces, 
                                                                                                                rfc_face_idxs, config_rhj_fc, 
                                                                                                                model_anatomical_path, nodal_o_dir)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-116.6246…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-116.6246…

minimum thickness in the initial layer with gap 0.92824
minimum thickness in the initial layer without gap 1.03138


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-116.6246…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-116.6246…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-91.17024…

### measure the gap between the two HJ cartilages

In [24]:
sd_value, _, closest_points = igl.signed_distance(rhj_fc_vertices_wg, rhj_ac_vertices_wg, rhj_ac_faces_wg, return_normals=False)

print('minimum distance in the left hip joint',np.round(np.min(sd_value),3))

# show the penetration
index = np.where (sd_value < 0 )[0]

frame = mp.plot(rhj_ac_vertices_wg, rhj_ac_faces_wg, c = src.pastel_orange, shading = src.sh_true)
frame.add_mesh (rhj_fc_vertices_wg, rhj_fc_faces_wg, c = src.pastel_yellow, shading = src.sh_true)

if len(index)!= 0 :
    frame.add_points(rhj_fc_vertices_wg[index], shading={"point_color": "green", "point_size": 20})
    

df = pd.read_csv(str(model_anatomical_path), encoding='utf-8')
df.loc[17, 'Value'] = np.round(np.min(sd_value),3)

df.to_csv(str(model_anatomical_path), index=False)

minimum distance in the left hip joint 0.099


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-86.66569…

In [25]:
rhj_fc_vertices_wog = src.remove_penetration(rhj_fc_vertices_wog, rhj_ac_vertices_wog, rhj_ac_faces_wog)

### export results

In [26]:
src.save_surface (rhj_fc_vertices_wg, rhj_fc_faces_wg, o_dim, o_rhj_fc_wg_path)
src.save_surface (rhj_ac_vertices_wg, rhj_ac_faces_wg, o_dim, o_rhj_ac_wg_path)

src.save_surface (rhj_fc_vertices_wog, rhj_fc_faces_wog, o_dim, o_rhj_fc_wog_path)
src.save_surface (rhj_ac_vertices_wog, rhj_ac_faces_wog, o_dim, o_rhj_ac_wog_path)

### right SI joint

In [27]:
config_rsj = config.rsj_var

if not full_model:
    config_rsj.full_model = False

In [28]:
smooth_s_vertices, smooth_rp_vertices, rsj_vertices, rsj_faces = src.get_sj(s_vertices, s_faces,
                                                                           smooth_rp_vertices, rp_faces,
                                                                           config_rsj, model_anatomical_path)

# export results
src.save_surface (rsj_vertices, rsj_faces, o_dim, o_rsj_path)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.207897…

Primary side: max dihedral angle before smoothing is  0.57 radians ( 32.66 degrees).
Primary side: max dihedral angle after smoothing is  0.57 radians ( 32.66 degrees).

Quality control results for the primary side: 
Everything is clean in the primary interface. We will now continue to the secondary interface:



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.205169…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-68.46923…

Secondary layer: max dihedral angle before smoothing is  0.58 radians ( 33.23 degrees).
Secondary layer: max dihedral angle after smoothing is  0.58 radians ( 33.23 degrees).
Quality control results for the base layer: 

Everything is clean in the base layer. We will now continue to create the wall:



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-68.46923…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.205169…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-68.46923…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-56.20844…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-56.20844…

### pubic cartilage

In [29]:
config_pj = config.pj_var

if not full_model:
    config_pj.full_model = False

In [30]:
# make it
smooth_lp_vertices, smooth_rp_vertices, pj_vertices, pj_faces = src.get_gap_pj(smooth_lp_vertices, lp_faces,
                                                                             smooth_rp_vertices, rp_faces,
                                                                             config_pj, model_anatomical_path)
  
# export results
src.save_surface ( pj_vertices, pj_faces, o_dim, o_pj_path )

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(60.993045…

Primary side: max dihedral angle before smoothing is  1.01 radians ( 57.87 degrees).
Primary side: max dihedral angle after smoothing is  1.01 radians ( 57.87 degrees).

Quality control results for the primary side: 
Everything is clean in the primary interface. We will now continue to the secondary interface:



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(60.993045…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-68.46923…

Secondary layer: max dihedral angle before smoothing is  0.88 radians ( 50.42 degrees).
Secondary layer: max dihedral angle after smoothing is  0.89 radians ( 50.99 degrees).
Quality control results for the base layer: 

Everything is clean in the base layer. We will now continue to create the wall:


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-68.46923…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(60.993045…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-68.46923…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-6.499807…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-6.499807…

## timer

In [31]:
# stop the time
now = time.time()
print("computation time: ", np.round(now-then, 2), 'seconds,', np.round((now-then)/60, 2), 'minutes')


computation time:  286.7 seconds, 4.78 minutes


## voila!

In [32]:
frame = mp.plot( smooth_s_vertices, s_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( smooth_lp_vertices, lp_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( smooth_rp_vertices, rp_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( smooth_lf_vertices, lf_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( smooth_rf_vertices, rf_faces, c = src.bone, shading = src.sh_false )
frame.add_mesh ( lhj_ac_vertices_wg, lhj_ac_faces_wg, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( lhj_fc_vertices_wg, lhj_fc_faces_wg, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( rhj_ac_vertices_wg, rhj_ac_faces_wg, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( rhj_fc_vertices_wg, rhj_fc_faces_wg, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( lsj_vertices, lsj_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( rsj_vertices, rsj_faces, c = src.pastel_light_blue, shading = src.sh_false )
frame.add_mesh ( pj_vertices, pj_faces, c = src.pastel_light_blue, shading = src.sh_false)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.205169…

11

## check self intersection in tetgen

In [33]:
# lsi_cart_test  = str((o_dir/ lsj).with_suffix('.stl'))
# rsi_cart_test  = str((o_dir/ rsj).with_suffix('.stl'))
# pubic_cart_test = str((o_dir/ pj).with_suffix('.stl'))

# # with gap 
# lpelvic_cart_test = str((o_dir/ lhj_ac_wg).with_suffix('.stl'))
# rpelvic_cart_test = str((o_dir/ rhj_ac_wg).with_suffix('.stl'))

# lfemoral_cart_test = str((o_dir/ lhj_fc_wg).with_suffix('.stl'))
# rfemoral_cart_test = str((o_dir/ rhj_fc_wg).with_suffix('.stl'))

# # withoutgap
# lpelvic_cart_test_wo_gap = str((o_dir/ lhj_ac_wog).with_suffix('.stl'))
# rpelvic_cart_test_wo_gap = str((o_dir/ rhj_ac_wog).with_suffix('.stl'))

# lfemoral_cart_test_wo_gap = str((o_dir/ lhj_fc_wog).with_suffix('.stl'))
# rfemoral_cart_test_wo_gap = str((o_dir/ rhj_fc_wog).with_suffix('.stl'))

# src.save_surface ( lsj_vertices, lsj_faces, o_dim, lsi_cart_test )
# src.save_surface ( rsj_vertices, rsj_faces, o_dim, rsi_cart_test )
# src.save_surface ( lhj_ac_vertices_wg, lhj_ac_faces_wg, o_dim, lpelvic_cart_test )
# src.save_surface ( rhj_ac_vertices_wg, rhj_ac_faces_wg, o_dim, rpelvic_cart_test )
# src.save_surface ( lhj_fc_vertices_wg, lhj_fc_faces_wg, o_dim, lfemoral_cart_test )
# src.save_surface ( rhj_fc_vertices_wg, rhj_fc_faces_wg, o_dim, rfemoral_cart_test )

# src.save_surface ( lhj_ac_vertices_wog, lhj_ac_faces_wog, o_dim, lpelvic_cart_test_wo_gap )
# src.save_surface ( rhj_ac_vertices_wog, rhj_ac_faces_wog, o_dim, rpelvic_cart_test_wo_gap )
# src.save_surface ( lhj_fc_vertices_wg, lhj_fc_faces_wg, o_dim, lfemoral_cart_test_wo_gap )
# src.save_surface ( rhj_fc_vertices_wg, rhj_fc_faces_wg, o_dim, rfemoral_cart_test_wo_gap )

# src.save_surface ( pj_vertices, pj_faces, o_dim, pubic_cart_test )

# os.system ( 'tetgen -d '+ lsi_cart_test )
# os.system ( 'tetgen -d '+ rsi_cart_test )
# os.system ( 'tetgen -d '+ lpelvic_cart_test )
# os.system ( 'tetgen -d '+ rpelvic_cart_test )
# os.system ( 'tetgen -d '+ lfemoral_cart_test )
# os.system ( 'tetgen -d '+ rfemoral_cart_test )
# os.system ( 'tetgen -d '+ pubic_cart_test )


# os.system ( 'tetgen -d '+ lpelvic_cart_test_wo_gap )
# os.system ( 'tetgen -d '+ rpelvic_cart_test_wo_gap )
# os.system ( 'tetgen -d '+ lfemoral_cart_test_wo_gap )
# os.system ( 'tetgen -d '+ rfemoral_cart_test_wo_gap )


## export bones 

In [34]:
src.save_surface ( smooth_s_vertices, s_faces, o_dim, o_sacrum_path ) 
src.save_surface ( smooth_lp_vertices, lp_faces, o_dim, o_lpelvis_path )
src.save_surface ( smooth_rp_vertices, rp_faces, o_dim, o_rpelvis_path )
src.save_surface ( smooth_lf_vertices, lf_faces, o_dim, o_lfemur_path )
src.save_surface ( smooth_rf_vertices, rf_faces, o_dim, o_rfemur_path )